In [23]:
import re
from utils.PromptOperator import PromptOperator

class BartPromptOperator(PromptOperator):
    POSITIVE_TEMPLATE = "{candidate_span}是一个{entity_type}实体"
    NEGATIVE_TEMPLATE = "{candidate_span}不是一个命名实体"
    LABEL_ENTITY = { "LOC": "地点", "ORG": "组织", "PER": "人名", "GPE": "地缘政治实体" }
    ENTITY_LABEL = { "地点": "LOC", "组织": "ORG", "人名": "PER", "地缘政治实体": "GPE" }

    def __init__(self, reader):
        super().__init__(reader)

    def __generate_golden_entity(self, sentence, label):
        golden_entity = []
        new_item, new_tag = None, None
        for item, tag in zip(sentence, label):
            if re.match(r"B-", tag):
                golden_entity.append([item, tag[2:]])
            elif re.match(r"I-", tag):
                golden_entity[-1][0] += item
        return list(map(lambda lst: (lst[0], lst[1]), golden_entity))

    def _format(self, sentence, label):
        result = []
        golden_entity = self.__generate_golden_entity(sentence, label)

        return ["".join(sentence)]

In [24]:
prompt_op = BartPromptOperator("./data/msra.min.dev")
prompt_op.dump("./test")